In [ ]:
import kagglehub
import pandas as pd
import numpy as np
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf

2025-06-09 21:18:21.190710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749493101.236800   44913 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749493101.250763   44913 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749493101.287015   44913 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749493101.287055   44913 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749493101.287061   44913 computation_placer.cc:177] computation placer alr

In [2]:
# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /home/eugen/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [3]:
dataset = pd.read_csv(path + '/IMDB Dataset.csv')
pd.set_option('display.max_colwidth', None)
dataset.head(3), dataset.info(), dataset.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


(                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [4]:
dataset = dataset.sample(n=2000)
dataset['review'] = dataset['review'].apply(lambda row: row.lower())
dataset['review'] = dataset['review'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
dataset.sample(3)

,review,sentiment
14105,i had high hopes for it when i heard that it was being made back in because i read the devil and daniel webster when i was a kid and i found it very interesting they made some changes to the story that dont make much sense to me daniel webster in the story was a famous lawyer from new hampshire in the story in the movie he is an editor a lawyer makes more sense since he ends up representing jabez stone against the devil himherself he was a man in the story but was a woman in the movie in a trial where both of their souls are on the line as an editor it doesnt seem likely that daniel webster would have the skill to do thisbr br the acting was decent by all except for alec baldwin and dan aykroyd these are two actors that i like they just did an awful job in this movie it was as though they thought they were acting in a comedy but the movie was more a serious one than a comedy this might be partly due to the fact that the movie was filmed with a particular vision in mind and was then reedited by somebody else given this fact its surprising that it was at all coherent i was surprised to see a fair amount of snl cast members in the movie which further leads me to believe it may have originally been filmed with the intention of it being more of a comedybr br all in all i would have to say it wasnt completely awful but it wasnt much good if i could get the hour and a half back and do something else with it i would the ending was especially disappointing as in the original story daniel webster defeats the devil in the trial jabez then starts out again at the beginning of the movieliterally we are just brought back to the first scene with jabez and then the movie abruptly ends it actually looked as though they just replayed jabez first scene over and called it the end there is no indication that jabez has the benefit of any of the knowledge or experience he gained so who is to say he didnt just repeat his mistakes over again and perhaps over and over in an endless loop it was an extremely disappointing end and did not make a lot of sense the decent cast and the acting of everyone except for baldwin and aykroyd are the only things that keep this from being a complete and total crap sandwich,negative
33617,warning slight plot spoilers aheadbr br the italian job is not the best movie youll see all year or probably even this summer but it is a worthwhile two hours because it colors within the lines knowing its limits and not attempting to exceed thembr br what carries the movie is the work of the cast in a movie about a crew of thieves the individuals must have a good rapport with each other without that cohesive feel the audience doesnt believe in the characters collectively or individually and the movie never has a chance but from the first scenes in which the men joke around and rag on each other while infiltrating a venetian palace the proper chemistry is in placebr br the characters themselves arent anything novel theyre your basic gang of criminals containing about half a dozen players each with a specific and defining skill but each actor brings the proper goods to the table for his or her part mark wahlbergs understated acting and humor fits well with his part as the mastermind planner edward norton provides attitude and twirls his mustache well in his dark role donald sutherland is the father figure of the crew and he looks the part of the suave and oldfashioned thief who is still mentally spry jason statham seth green and mos def dont do much beyond their characters abilities but they each nail those parts statham as the smoothoperating driver green as the tech whiz geek with a chip on his shoulder and def as the demolitions man charlize theron slides in well in a part that doesnt ask too much of her she is primarily asked to to drive fast and look good that she does none of the characters are that deep or threedimensional but in this familiar sort of movie two dimensions are all that is required br br as

In [5]:
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')

def tokenize_text(text):
    return tokenizer(
         text.tolist(),
         max_length = 50,
         truncation = True,
         padding = 'max_length',
         return_tensors = 'np'
    )

tokenized = tokenize_text(dataset['review'])
tokenized

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

{'input_ids': array([[  101,  6511,  7380, ...,  4038,  1998,   102],
       [  101,  2021,  1996, ..., 28667, 18557,   102],
       [  101,  1045,  2245, ...,  2003,  8857,   102],
       ...,
       [  101,  1045, 12246, ..., 26749,  2024,   102],
       [  101,  2065,  2412, ...,  4121, 20014,   102],
       [  101,  6195,  2035, ...,  2054,  2500,   102]], shape=(2000, 50)), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(2000, 50)), 'attention_mask': array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], shape=(2000, 50))}

In [8]:
bert_model = TFBertModel.from_pretrained('prajjwal1/bert-tiny', from_pt=True)
outputs = bert_model(tokenized)

X = outputs.last_hidden_state[:, 0, :].numpy()
y = dataset['sentiment']


Loading a PyTorch model in TensorFlow, requires both PyTorch and TensorFlow to be installed. Please see https://pytorch.org/ and https://www.tensorflow.org/install/ for installation instructions.


ModuleNotFoundError: No module named 'torch'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import spacy

spacy_nlp = spacy.load("en_core_web_sm")

def extract_actor_sentiment(text):
    doc = spacy_nlp(text)
    actor_sentiment = None
    for token in doc:
        if token.lemma_ == "actor" or token.lemma_ == "actors" and token.pos_ == "NOUN":
            for child in token.children:
                if child.pos_ == "ADJ":
                    sentiment = child.text.lower()
                    if sentiment in ['good', 'great', 'excellent', 'amazing', 'brilliant', 'fantastic']:
                        actor_sentiment = 0
                    elif sentiment in ['bad', 'terrible', 'awful', 'poor', 'horrible']:
                        actor_sentiment = 1
                    break
            if actor_sentiment:
                break
    return actor_sentiment


dataset['actor_sentiment'] = dataset['review'].apply(extract_actor_sentiment)



In [ ]:
dataset_with_actors_sent = dataset[dataset['actor_sentiment'].notna()]
dataset_with_actors_sent.info()

In [ ]:
X_by_actor = dataset_with_actors_sent.drop('actor_sentiment', axis=1)
y_by_actor = dataset_with_actors_sent['actor_sentiment']

tokenized = tokenize_text(X_by_actor['review'])
outputs_by_actor = bert_model(tokenized)
X_by_actor = outputs_by_actor.last_hidden_state[:, 0, :].numpy()

X_train_actor, X_test_actor, y_train_actor, y_test_actor = train_test_split(X_by_actor, y_by_actor, test_size = 0.2)

log_reg_by_actor = LogisticRegression(max_iter=1000)
log_reg_by_actor.fit(X_train_actor, y_train_actor)
y_pred_actor = log_reg_by_actor.predict(X_test_actor)
print(classification_report(y_test_actor, y_pred_actor))